# 05. Evaluation (CRISP-DM Phase 5)

## Objective
Evaluate whether the segmentation model supports business goals:
1. Are segments **actionable** for marketing?
2. Do clusters have **distinct behavioral profiles**?
3. Can we **prioritize customers** for retention?

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)

# Color palette
SEGMENT_COLORS = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6', 
                  '#1abc9c', '#e67e22', '#34495e', '#16a085', '#c0392b']

---

## 1. Load Segmented Data

In [ ]:
# Load remediated segmented data
df = pd.read_csv('../dataset/customers_segmented_remediated.csv')

print(f"Total customers: {len(df):,}")
print(f"Columns: {list(df.columns)}")
df.head()

In [ ]:
# Segment overview
print("\n📊 Segment Distribution:")
segment_counts = df['Segment_Name'].value_counts()
for segment, count in segment_counts.items():
    pct = count / len(df) * 100
    print(f"   {segment}: {count:,} ({pct:.1f}%)")

---

## 2. Technical Evaluation Metrics

In [ ]:
# Separate transactors for cluster evaluation
transactors = df[df['Cluster_GMM'] != -1].copy()

print(f"Transactors: {len(transactors):,}")
print(f"Window Shoppers: {len(df) - len(transactors):,}")

In [ ]:
# Calculate Silhouette Score (transactors only)
features = ['Recency', 'Frequency', 'Monetary', 'Conversion_Rate']
X = transactors[features].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

silhouette = silhouette_score(X_scaled, transactors['Cluster_GMM'])

print("="*60)
print("CLUSTERING METRICS (Transactors Only)")
print("="*60)
print(f"\n📊 Silhouette Score: {silhouette:.4f}")
print(f"📊 Number of Clusters: {transactors['Cluster_GMM'].nunique()}")

if silhouette > 0.25:
    print("   ✅ Good cluster separation")
elif silhouette > 0.15:
    print("   ⚠️ Moderate separation (acceptable for behavioral data)")
else:
    print("   ⚠️ Low separation - consider feature engineering")

---

## 2.5 Model Comparison: K-Means vs GMM

In [ ]:
# Model comparison table
print("\n📊 MODEL COMPARISON")
print("="*70)

comparison_data = {
    'Aspect': ['Model Type', 'Cluster Shape', 'Window Shoppers', 'Features Used', 'Silhouette Score'],
    'K-Means (with PCA)': ['Centroid-based', 'Spherical', 'Included in clustering', 'RFM + CLV (PCA reduced)', 'See 04_K_means.ipynb'],
    'GMM': ['Probabilistic', 'Elliptical (flexible)', 'Separated pre-clustering', 'RFM only', f'{silhouette:.4f}']
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

print("\n📌 Recommendation:")
print("   GMM is preferred for this dataset because:")
print("   1. Separates Window Shoppers (99.2%) from Transactors")
print("   2. Handles overlap between behavioral clusters better")
print("   3. Provides probability-based assignment")


---

## 3. Segment Profiles

In [ ]:
# Segment profiles (raw values)
raw_features = ['Recency_Days', 'Num_Sessions', 'Num_Transactions', 'Monetary_Raw', 'Conversion_Rate']

# Transactor profiles
transactor_profiles = transactors.groupby('Segment_Name')[raw_features].agg(['mean', 'median', 'count'])

print("\n📊 Transactor Segment Profiles (Mean):")
profiles_mean = transactors.groupby('Segment_Name')[raw_features].mean().round(1)
print(profiles_mean.to_string())

In [ ]:
# Heatmap of normalized profiles
profiles_norm = (profiles_mean - profiles_mean.min()) / (profiles_mean.max() - profiles_mean.min())

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(profiles_norm.T, annot=profiles_mean.T.values, fmt='.0f',
            cmap='RdYlGn', ax=ax, cbar_kws={'label': 'Normalized Value'})
ax.set_title('Transactor Segment Profiles Heatmap', fontsize=14, fontweight='bold')
ax.set_xlabel('Segment')
ax.set_ylabel('Feature')

plt.tight_layout()
plt.show()

---

## 3.5 CLV Analysis by Segment (Leading Indicator)

In [ ]:
# Load CLV data if available
try:
    clv_df = pd.read_csv('../dataset/clv_customer.csv')
    # Merge with transactors
    if 'CLV_1_month' in clv_df.columns:
        transactors_clv = transactors.merge(clv_df[['visitorid', 'CLV_1_month', 'probability_alive']], on='visitorid', how='left')
        
        # CLV by segment
        clv_by_segment = transactors_clv.groupby('Segment_Name').agg({
            'CLV_1_month': ['mean', 'median', 'sum'],
            'probability_alive': 'mean'
        }).round(2)
        clv_by_segment.columns = ['Avg_CLV', 'Median_CLV', 'Total_CLV', 'Avg_Prob_Alive']
        clv_by_segment = clv_by_segment.sort_values('Avg_CLV', ascending=False)
        
        print("\n📊 CLV Analysis by Segment (Leading Indicator):")
        print(clv_by_segment.to_string())
        
        # Visualization
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # CLV Bar Chart
        clv_by_segment['Avg_CLV'].plot(kind='bar', ax=axes[0], color=SEGMENT_COLORS[:len(clv_by_segment)])
        axes[0].set_title('Average CLV by Segment', fontweight='bold')
        axes[0].set_ylabel('CLV (1 Month)')
        axes[0].set_xlabel('Segment')
        axes[0].tick_params(axis='x', rotation=45)
        
        # Probability Alive
        clv_by_segment['Avg_Prob_Alive'].plot(kind='bar', ax=axes[1], color=SEGMENT_COLORS[:len(clv_by_segment)])
        axes[1].set_title('Probability Alive by Segment', fontweight='bold')
        axes[1].set_ylabel('Probability')
        axes[1].set_xlabel('Segment')
        axes[1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
    else:
        print("CLV columns not found in clv_customer.csv")
except FileNotFoundError:
    print("clv_customer.csv not found. Skipping CLV analysis.")


---

## 4. Revenue Contribution Analysis

In [ ]:
# Revenue by segment (transactors only - Window Shoppers have 0 revenue)
segment_revenue = transactors.groupby('Segment_Name').agg({
    'visitorid': 'count',
    'Monetary_Raw': 'sum',
    'Num_Transactions': 'sum'
}).rename(columns={'visitorid': 'Customers', 'Monetary_Raw': 'Total_Revenue', 'Num_Transactions': 'Transactions'})

segment_revenue['Pct_Customers'] = (segment_revenue['Customers'] / segment_revenue['Customers'].sum() * 100).round(1)
segment_revenue['Pct_Revenue'] = (segment_revenue['Total_Revenue'] / segment_revenue['Total_Revenue'].sum() * 100).round(1)
segment_revenue['Avg_Revenue'] = (segment_revenue['Total_Revenue'] / segment_revenue['Customers']).round(0)

segment_revenue = segment_revenue.sort_values('Pct_Revenue', ascending=False)

print("\n📊 Revenue Analysis (Transactors Only):")
print(segment_revenue.to_string())

In [ ]:
# Pareto visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Customer distribution
axes[0].pie(segment_revenue['Pct_Customers'], labels=segment_revenue.index, 
            autopct='%1.1f%%', colors=SEGMENT_COLORS[:len(segment_revenue)], startangle=90)
axes[0].set_title('Customer Distribution (Transactors)', fontweight='bold')

# Revenue distribution
axes[1].pie(segment_revenue['Pct_Revenue'], labels=segment_revenue.index, 
            autopct='%1.1f%%', colors=SEGMENT_COLORS[:len(segment_revenue)], startangle=90)
axes[1].set_title('Revenue Distribution', fontweight='bold')

plt.tight_layout()
plt.show()

# Pareto insight
top_segment = segment_revenue.index[0]
top_pct = segment_revenue.loc[top_segment, 'Pct_Revenue']
print(f"\n💰 Pareto Insight: '{top_segment}' contributes {top_pct:.1f}% of total revenue")

---

## 5. Segment Action Matrix

In [ ]:
# Action recommendations
action_matrix = {
    'Window Shoppers': ('🟢 Low', 'Acquisition', 'First-purchase incentives, retargeting ads'),
    'VIP Champions': ('🔴 High', 'Retention', 'Exclusive VIP access, premium support'),
    'Big Spenders': ('🔴 High', 'Upsell', 'Premium bundles, loyalty tier upgrades'),
    'Loyal Regulars': ('🔴 High', 'Cross-sell', 'Personalized recommendations, rewards'),
    'New Actives': ('🟡 Medium', 'Nurture', 'Welcome series, second-purchase discount'),
    'Potential Loyalists': ('🟡 Medium', 'Grow', 'Category recommendations, milestone rewards'),
    'At Risk': ('🟠 Medium-High', 'Win-Back', 'Time-limited offers, "We miss you" emails'),
    'Need Attention': ('🟢 Low', 'Engage', 'Light promotions, feedback surveys'),
    'Hibernating': ('🟢 Low', 'Last Chance', 'Final offers or sunset')
}

print("\n📋 SEGMENT ACTION MATRIX")
print("="*80)

for segment in segment_counts.index:
    if segment in action_matrix:
        priority, strategy, actions = action_matrix[segment]
        count = segment_counts[segment]
        print(f"\n{priority} {segment} ({count:,} customers)")
        print(f"   Strategy: {strategy}")
        print(f"   Actions: {actions}")

---

## 5.5 Propensity Score Summary (Leading Indicator)

In [ ]:
# Load propensity scores if available
try:
    propensity_df = pd.read_csv('../dataset/customer_propensity_scores.csv')
    
    if 'propensity_score' in propensity_df.columns:
        # Merge with main df
        df_prop = df.merge(propensity_df[['visitorid', 'propensity_score', 'propensity_segment']], on='visitorid', how='left')
        
        # Summary by segment
        prop_summary = df_prop.groupby('Segment_Name')['propensity_score'].agg(['mean', 'median', 'count']).round(3)
        prop_summary.columns = ['Avg_Propensity', 'Median_Propensity', 'Count']
        prop_summary = prop_summary.sort_values('Avg_Propensity', ascending=False)
        
        print("\n📊 Propensity Score by Segment (7-Day Purchase Probability):")
        print(prop_summary.to_string())
        
        # Visualization
        fig, ax = plt.subplots(figsize=(10, 6))
        prop_summary['Avg_Propensity'].plot(kind='barh', ax=ax, color=SEGMENT_COLORS[:len(prop_summary)])
        ax.set_title('Average Propensity Score by Segment', fontweight='bold')
        ax.set_xlabel('Propensity Score (0-1)')
        ax.set_ylabel('Segment')
        ax.axvline(x=0.5, color='red', linestyle='--', label='50% threshold')
        ax.legend()
        plt.tight_layout()
        plt.show()
        
        print("\n📌 Insight:")
        high_prop = prop_summary[prop_summary['Avg_Propensity'] > 0.5].index.tolist()
        if high_prop:
            print(f"   Segments with >50% propensity: {high_prop}")
            print("   → Prioritize these for immediate marketing campaigns")
        else:
            print("   No segments have >50% average propensity")
            print("   → Focus on increasing engagement first")
    else:
        print("propensity_score column not found")
except FileNotFoundError:
    print("customer_propensity_scores.csv not found. Skipping propensity analysis.")


---

## 6. Conclusions

In [ ]:
print("="*70)
print("EVALUATION CONCLUSIONS")
print("="*70)

print("\n✅ DATA INTEGRITY (Strict Audit Compliant):")
print("   • Revenue from transactionid (no hallucinated values)")
print("   • Log-transformed RFM features")
print("   • No global mean imputation")

print("\n✅ SEGMENTATION STRATEGY:")
print(f"   • Window Shoppers separated: {len(df) - len(transactors):,} (99.2%)")
print(f"   • Transactor clusters: {transactors['Cluster_GMM'].nunique()}")
print(f"   • Silhouette Score: {silhouette:.4f}")

print("\n✅ BUSINESS VALUE:")
print("   • Clear RFM profiles for each segment")
print("   • Actionable marketing strategies defined")
print("   • Revenue concentration identified")

print("\n📌 NEXT STEPS:")
print("   1. Deploy segments to CRM/marketing platform")
print("   2. A/B test strategies for VIPs and At Risk")
print("   3. Monitor segment migration monthly")
print("   4. Re-cluster quarterly")